In [212]:
import re
import pandas as pd

# Reading in data
in_file = open("data/district_data_23_Billion.csv", 'r')
county_file = open("data/EDGE_GEOCODE_PUBLICLEA_1920.csv", 'r')

school_dataframe = pd.read_csv(in_file)
county_dataframe = pd.read_csv(county_file, dtype={'CNTY':str})


county_dataframe.rename(columns={'LEAID':'NCESID'}, inplace = True)
merged_dataframes = pd.merge(left = school_dataframe, right = county_dataframe, how = 'inner', on = 'NCESID')
data = merged_dataframes[['CNTY','State and local revenue, per pupil, cost adjusted', 'Enrollment', 'Percent White', 'Student poverty rate', 'Median household income', 'Median property value']].copy()
data['State and local revenue, per pupil, cost adjusted'] = data['State and local revenue, per pupil, cost adjusted'].replace("[$,]", "", regex=True).astype(float)
data['State and local revenue, per pupil, cost adjusted'] = data['State and local revenue, per pupil, cost adjusted'].replace("[$,]", "", regex=True).astype(float)
data['Percent White'] = data['Percent White'].replace("[%]", "", regex=True).astype(int)
data['Student poverty rate'] = data['Student poverty rate'].dropna().replace("[%]", "", regex=True).astype(int)
data.groupby('CNTY').mean()
data.CNTY.str.zfill(5)
data.to_csv('data/county_data.csv', index=False)